<a href="https://colab.research.google.com/github/xd3011/DKML_ORTOOLS_ILP/blob/main/examples/notebook/sat/cp_sat_example_min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# cp_sat_example

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/main/examples/notebook/sat/cp_sat_example.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/main/ortools/sat/samples/cp_sat_example.py"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [5]:
%pip install ortools


Simple solve.

In [6]:
from ortools.linear_solver import pywraplp
import math

# =============================================================================
#  PARAMETERS globales (GLOBAL PARAMETERS)
# =============================================================================
# Các hằng số này giúp dễ dàng điều chỉnh "luật chơi" của bài toán
# mà không cần sửa logic bên trong các hàm.

# Số học viên tối thiểu để mở một lớp
MIN_STUDENT_PER_CLASS = 3

# Trọng số cơ bản khi xếp được một sinh viên vào lớp
STUDENT_BASE_WEIGHT = 100

# Điểm cộng cho mỗi buổi học sinh viên đã lỡ (khuyến khích xếp lớp cho SV này)
MISSED_SESSION_WEIGHT = 5

# Điểm cộng dựa trên số session đã đăng ký (ưu tiên SV đăng ký ít)
# Dùng 1/số session để SV nào đăng ký càng ít thì điểm càng cao
REGISTERED_SESSIONS_WEIGHT = 20

# Điểm phạt khi mở một lớp mới
CLASS_OPEN_PENALTY = -150

# Hệ số phạt cho việc sử dụng phòng lớn (khuyến khích dùng phòng vừa đủ)
# Giá trị càng lớn, solver càng "ngại" dùng phòng to.
# Nên nhỏ hơn nhiều so với STUDENT_BASE_WEIGHT.
CAPACITY_PENALTY_FACTOR = 1

# =============================================================================
# Các hàm phụ trợ (HELPER FUNCTIONS)
# =============================================================================

def slots_conflict(slot_a, slot_b):
    """Kiểm tra 2 slot có trùng nhau không."""
    return not (slot_a["end"] <= slot_b["begin"] or slot_b["end"] <= slot_a["begin"])


def room_busy_conflict(room_busy_slots, class_slot):
    """Kiểm tra slot có trùng với lịch bận của phòng."""
    return any(slots_conflict(busy_slot, class_slot) for busy_slot in room_busy_slots)


# =============================================================================
# Các hàm chính của mô hình (CORE MODEL FUNCTIONS)
# =============================================================================

def create_decision_variables(solver, data):
    sessions, rooms, students, classes_by_session = data
    """Tạo tất cả các biến quyết định cho bài toán."""
    print("\n=== 🔧 Tạo biến quyết định ===")
    is_class_open = {
        class_id: solver.BoolVar(f"is_class_open_{class_id}")
        for session_id in sessions
        for class_id in classes_by_session[session_id]
    }

    is_room_assigned_to_class = {
        (class_id, room_id): solver.BoolVar(f"is_room_{room_id}_for_{class_id}")
        for session_id, session in sessions.items()
        for class_id in classes_by_session[session_id]
        for room_id, room in rooms.items()
        if room["facility"] == session["facility"]
    }

    is_student_assigned_to_class = {
        (student_id, class_id): solver.BoolVar(f"is_student_{student_id}_in_{class_id}")
        for session_id, session in sessions.items()
        for class_id in classes_by_session[session_id]
        for student_id in session["registers"]
    }

    print(f"Số biến is_class_open: {len(is_class_open)}")
    print(f"Số biến is_room_assigned_to_class: {len(is_room_assigned_to_class)}")
    print(f"Số biến is_student_assigned_to_class: {len(is_student_assigned_to_class)}")

    return is_class_open, is_room_assigned_to_class, is_student_assigned_to_class


def add_constraints(solver, data, variables):
    """Thêm tất cả các ràng buộc vào solver."""
    print("\n=== 📏 Thêm ràng buộc ===")
    sessions, rooms, students, classes_by_session = data
    is_class_open, is_room_assigned, is_student_assigned = variables

    # [1] Nếu SV được xếp thì lớp phải mở
    for student_id, class_id in is_student_assigned:
        solver.Add(is_student_assigned[student_id, class_id] <= is_class_open[class_id])

    # [2] Học viên chỉ học 1 lớp của mỗi session
    for session_id, session in sessions.items():
        for student_id in session["registers"]:
            solver.Add(solver.Sum(is_student_assigned[student_id, c_id] for c_id in classes_by_session[session_id]) <= 1)

    # [3] Nếu lớp mở => phải chọn đúng 1 phòng
    for class_list in classes_by_session.values():
        for class_id in class_list:
            possible_rooms = [is_room_assigned[class_id, r_id] for r_id in rooms if (class_id, r_id) in is_room_assigned]
            solver.Add(solver.Sum(possible_rooms) == is_class_open[class_id])

    # [4] Mỗi phòng chỉ có 1 lớp trong cùng thời gian
    for room_id in rooms:
        class_slots = []
        for session_id, session in sessions.items():
            if (list(classes_by_session[session_id])[0], room_id) in is_room_assigned:
                for class_id in classes_by_session[session_id]:
                    class_slots.append((class_id, session["slots"]))
        for i in range(len(class_slots)):
            for j in range(i + 1, len(class_slots)):
                class_a, slots_a = class_slots[i]
                class_b, slots_b = class_slots[j]
                if any(slots_conflict(sa, sb) for sa in slots_a for sb in slots_b):
                    solver.Add(is_room_assigned[class_a, room_id] + is_room_assigned[class_b, room_id] <= 1)

    # [5] Không trùng lịch bận phòng
    for session_id, session in sessions.items():
        for class_id in classes_by_session[session_id]:
            for room_id, room in rooms.items():
                if (class_id, room_id) in is_room_assigned:
                    for slot in session["slots"]:
                        if room_busy_conflict(room["busy"], slot):
                            solver.Add(is_room_assigned[class_id, room_id] == 0)

    # [6] Số học viên tối thiểu để mở lớp
    for session_id, session in sessions.items():
        for class_id in classes_by_session[session_id]:
            num_students = solver.Sum(is_student_assigned[s_id, class_id] for s_id in session["registers"])
            solver.Add(num_students >= MIN_STUDENT_PER_CLASS * is_class_open[class_id])

    # [7] Sức chứa phòng
    for session_id, session in sessions.items():
        students_in_session = session["registers"]
        big_m = len(students_in_session)
        for class_id in classes_by_session[session_id]:
            num_students_in_class = solver.Sum(is_student_assigned[s_id, class_id] for s_id in students_in_session)
            for room_id, room in rooms.items():
                if (class_id, room_id) in is_room_assigned:
                    solver.Add(num_students_in_class <= room["capacity"] + big_m * (1 - is_room_assigned[class_id, room_id]))

    # [8] Không cho SV học trùng giờ khác môn
    sessions_by_student = {s_id: [ss_id for ss_id, ss in sessions.items() if s_id in ss["registers"]] for s_id in students}
    for student_id, session_list in sessions_by_student.items():
        for i in range(len(session_list)):
            for j in range(i + 1, len(session_list)):
                s_a_id, s_b_id = session_list[i], session_list[j]
                session_a, session_b = sessions[s_a_id], sessions[s_b_id]
                if session_a["subject"] != session_b["subject"] and any(slots_conflict(s_a, s_b) for s_a in session_a["slots"] for s_b in session_b["slots"]):
                    for class_a in classes_by_session[s_a_id]:
                        for class_b in classes_by_session[s_b_id]:
                            solver.Add(is_student_assigned[student_id, class_a] + is_student_assigned[student_id, class_b] <= 1)


def define_objective(solver, data, variables):
    """Xây dựng hàm mục tiêu."""
    print("\n=== 🎯 Xây dựng hàm mục tiêu ===")
    sessions, rooms, students, classes_by_session = data
    is_class_open, is_room_assigned, is_student_assigned = variables

    objective = solver.Objective()

    # Thêm điểm thưởng/phạt vào hàm mục tiêu
    for class_id, var in is_class_open.items():
        objective.SetCoefficient(var, CLASS_OPEN_PENALTY)

    for (class_id, room_id), var in is_room_assigned.items():
        penalty = -1 * CAPACITY_PENALTY_FACTOR * rooms[room_id]['capacity']
        objective.SetCoefficient(var, penalty)

    for (student_id, class_id), var in is_student_assigned.items():
        session_id = class_id.split('_C')[0]
        session = sessions[session_id]
        student_info = students[student_id]
        weight = STUDENT_BASE_WEIGHT
        weight += REGISTERED_SESSIONS_WEIGHT * (1 / student_info["registered_sessions"])
        weight += MISSED_SESSION_WEIGHT * student_info["missed"].get(session["subject"], 0)
        objective.SetCoefficient(var, weight)

    objective.SetMaximization()

def print_decision_analysis(data, variables):
    """
    In ra một báo cáo chi tiết cho TỪNG lớp tiềm năng.
    - Phân biệt rõ lý do đóng lớp: Kinh tế, Sĩ số, Lớp thay thế, hoặc Cạnh tranh tài nguyên.
    """
    print("\n" + "="*80)
    print("📊 BÁO CÁO PHÂN TÍCH QUYẾT ĐỊNH CHI TIẾT CHO TỪNG LỚP".center(80))
    print("="*80 + "\n")

    sessions, rooms, students, classes_by_session = data
    is_class_open, is_room_assigned, is_student_assigned = variables

    for session_id, session in sessions.items():
        for class_id in classes_by_session[session_id]:

            print(f"--- Lớp: {class_id} ({session['subject']}) ---")

            if is_class_open[class_id].solution_value() > 0.5:
                # KỊCH BẢN 1: LỚP ĐƯỢC MỞ (Phân tích dựa trên kết quả thực tế)
                assigned_students_count = 0
                total_student_reward = 0
                for student_id in session["registers"]:
                    if is_student_assigned[student_id, class_id].solution_value() > 0.5:
                        assigned_students_count += 1
                        student_info = students[student_id]
                        weight = STUDENT_BASE_WEIGHT \
                                 + REGISTERED_SESSIONS_WEIGHT * (1 / student_info["registered_sessions"]) \
                                 + MISSED_SESSION_WEIGHT * student_info["missed"].get(session["subject"], 0)
                        total_student_reward += weight

                room_id_assigned = next(r_id for (c_id, r_id), var in is_room_assigned.items()
                                        if c_id == class_id and var.solution_value() > 0.5)
                room_capacity = rooms[room_id_assigned]['capacity']
                class_cost = CLASS_OPEN_PENALTY
                net_value = total_student_reward + class_cost

                print(f"  - Trạng thái: ✅ ĐƯỢC MỞ")
                print(f"  - Sĩ số:      {assigned_students_count} sinh viên (Yêu cầu tối thiểu: {MIN_STUDENT_PER_CLASS})")
                print(f"  - Phòng học:   {room_id_assigned} (Sức chứa: {room_capacity})")
                print("  - Phân tích kinh tế (Thực tế):")
                print(f"    - Lợi ích từ SV: {total_student_reward:,.0f}")
                print(f"    - Chi phí mở lớp:   {class_cost:,.0f}")
                print(f"    - Chênh lệch:       {net_value:,.0f} (Dương -> Có lời)")

            else:
                # KỊCH BẢN 2: LỚP BỊ ĐÓNG (Phân tích dựa trên kịch bản giả định)
                print(f"  - Trạng thái: ❌ BỊ ĐÓNG")

                potential_student_count = len(session["registers"])
                potential_reward = 0
                for student_id in session["registers"]:
                    student_info = students[student_id]
                    weight = STUDENT_BASE_WEIGHT \
                             + REGISTERED_SESSIONS_WEIGHT * (1 / student_info["registered_sessions"]) \
                             + MISSED_SESSION_WEIGHT * student_info["missed"].get(session["subject"], 0)
                    potential_reward += weight

                class_cost = CLASS_OPEN_PENALTY
                net_value = potential_reward + class_cost

                print(f"  - Sĩ số tiềm năng: {potential_student_count} sinh viên (Yêu cầu tối thiểu: {MIN_STUDENT_PER_CLASS})")
                print("  - Phân tích kinh tế (Giả định):")
                print(f"    - Lợi ích tiềm năng tối đa: {potential_reward:,.0f}")
                print(f"    - Chi phí mở lớp:          {class_cost:,.0f}")
                print(f"    - Chênh lệch tiềm năng:     {net_value:,.0f}")

                print("  - Phân tích lý do:")
                if potential_student_count < MIN_STUDENT_PER_CLASS:
                    print("    - Lý do chính: [RÀNG BUỘC CỨNG] Sĩ số đăng ký không đạt mức tối thiểu.")
                elif net_value < 0:
                    print("    - Lý do chính: [KINH TẾ] Lợi ích tiềm năng tối đa vẫn không đủ để bù đắp chi phí.")
                else:
                    # **LOGIC MỚI ĐỂ PHÂN TÍCH SÂU HƠN**
                    # Kiểm tra xem có lớp thay thế nào đã được mở không
                    substitute_class_found = None
                    current_session_id = class_id.split('_C')[0]
                    for other_class_id in classes_by_session[current_session_id]:
                        if other_class_id != class_id and is_class_open[other_class_id].solution_value() > 0.5:
                            substitute_class_found = other_class_id
                            break

                    if substitute_class_found:
                        print(f"    - Lý do chính: [LỚP THAY THẾ] Nhu cầu cho session này đã được đáp ứng bởi lớp")
                        print(f"      '{substitute_class_found}', do đó không cần mở thêm lớp này để tránh lãng phí.")
                    else:
                        print("    - Lý do chính: [CẠNH TRANH TÀI NGUYÊN] Lớp này có lời, nhưng sinh viên hoặc phòng")
                        print("      đã được dùng cho các lựa chọn ở MÔN HỌC KHÁC mang lại tổng điểm Z cao hơn.")

            print("-" * (len(class_id) + 12) + "\n")

def solve_and_print_results(solver, data, variables, students):
    """Giải bài toán và in kết quả."""
    print("\n=== 🚀 Bắt đầu giải bài toán ===")
    sessions, rooms, _, classes_by_session = data
    is_class_open, is_room_assigned, is_student_assigned = variables

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("\n✅ Lời giải tối ưu đã được tìm thấy.")
        print_decision_analysis(data, variables)
        print(f"🏆 Giá trị hàm mục tiêu Z (Tổng điểm): {solver.Objective().Value()}")
        print_statistics(sessions, students, classes_by_session, is_class_open, is_student_assigned)
        print("\n Danh sách các lớp có thể mở: \n")
        for session_id, session in sessions.items():
            for class_id in classes_by_session[session_id]:
                if is_class_open[class_id].solution_value() > 0.5:
                    room_id = next(r_id for (c_id, r_id), var in is_room_assigned.items() if c_id == class_id and var.solution_value() > 0.5)
                    assigned_students = [s_id for (s_id, c_id), var in is_student_assigned.items() if c_id == class_id and var.solution_value() > 0.5]

                    print(f"📌 Lớp {class_id} | Phòng: {room_id} "
                          f"| Sĩ số: {len(assigned_students)}/{rooms[room_id]['capacity']} "
                          f"| Học viên: {assigned_students}")
    else:
        print("\n❌ Không tìm thấy lời giải tối ưu.")
    print("\n=== Các tham số điều chỉnh bài toán ===\n")
    print(f"Số học viên tối thiểu để mở một lớp (MIN_STUDENT_PER_CLASS): {MIN_STUDENT_PER_CLASS}")
    print(f"Trọng số cơ bản khi xếp được một sinh viên vào lớp (STUDENT_BASE_WEIGHT): {STUDENT_BASE_WEIGHT}")
    print(f"Điểm cộng cho mỗi buổi học sinh viên đã lỡ (MISSED_SESSION_WEIGHT): {MISSED_SESSION_WEIGHT}")
    print(f"Điểm cộng dựa trên số session đã đăng ký (REGISTERED_SESSIONS_WEIGHT): {REGISTERED_SESSIONS_WEIGHT}")
    print(f"Điểm phạt khi mở một lớp mới (CLASS_OPEN_PENALTY): {CLASS_OPEN_PENALTY}")
    print(f"Hệ số phạt cho việc sử dụng phòng lớn (CAPACITY_PENALTY_FACTOR): {CAPACITY_PENALTY_FACTOR}")
    print("\n=== 🏁 Kết thúc giải bài toán ===")

def print_statistics(sessions, students, classes_by_session, is_class_open, is_student_assigned):
    """In thông tin thống kê từ kết quả giải."""
    print("\n=== 📊 Thống kê kết quả ===")

    # Sessions khả thi (tất cả sessions trong dữ liệu đầu vào)
    num_possible_sessions = len(sessions)
    print(f"- Sessions khả thi: {num_possible_sessions}")

    # Số Sinh viên được phục vụ
    assigned_students_set = set()
    for (student_id, class_id), var in is_student_assigned.items():
        if var.solution_value() > 0.5:
            assigned_students_set.add(student_id)
    num_assigned_students = len(assigned_students_set)
    print(f"- Số Sinh viên được phục vụ: {num_assigned_students}")

    # Lớp được mở
    opened_classes = [class_id for class_id, var in is_class_open.items() if var.solution_value() > 0.5]
    num_opened_classes = len(opened_classes)
    print(f"- Lớp được mở: {num_opened_classes}")

    # Các sinh viên không được phục vụ
    all_students = set(students.keys())
    unassigned_students = all_students - assigned_students_set
    print(f"- Các sinh viên không được phục vụ: {list(unassigned_students)}")


def run_scheduler(sessions, rooms, students):
    """Hàm chính điều phối toàn bộ quá trình."""
    print("=== 🔍 Khởi tạo solver ===")
    solver = pywraplp.Solver.CreateSolver("SCIP")

    # Tạo danh sách các lớp học tiềm năng
    print("\n=== 🏫 Tạo danh sách lớp cho mỗi session ===")
    classes_by_session = {}
    for session_id, session in sessions.items():
        max_c = math.floor(len(session["registers"]) / MIN_STUDENT_PER_CLASS) or 1
        classes_by_session[session_id] = [f"{session_id}_C{i}" for i in range(max_c)]
        print(f"Session {session_id} -> Lớp: {classes_by_session[session_id]}")

    # Đóng gói dữ liệu để truyền cho các hàm
    data = (sessions, rooms, students, classes_by_session)

    # 1. Tạo biến quyết định
    variables = create_decision_variables(solver, data)

    # 2. Thêm các ràng buộc
    add_constraints(solver, data, variables)

    # 3. Xây dựng hàm mục tiêu
    define_objective(solver, data, variables)

    # 4. Giải và in kết quả
    solve_and_print_results(solver, data, variables, students)


def main():
    sessions = {
        "SS1": {
            "subject": "Math",
            "facility": "NEU",
            "slots": [{"begin": 8, "end": 10}, {"begin": 14, "end": 16}],
            "registers": ["alice", "bob", "charlie", "david", "eva", "frank", "grace"]
        },
        "SS2": {
            "subject": "Physics",
            "facility": "NEU",
            "slots": [{"begin": 10, "end": 12}],
            "registers": ["alice", "bob", "charlie", "david", "eva"]
        },
        "SS3": {
            "subject": "English",
            "facility": "FTU",
            "slots": [{"begin": 9, "end": 11}, {"begin": 13, "end": 15}],
            "registers": ["alice", "charlie", "henry", "iris", "jack", "kate"]
        },
        "SS4": {
            "subject": "Chemistry",
            "facility": "FTU",
            "slots": [{"begin": 15, "end": 17}],
            "registers": ["bob", "david", "henry", "iris"]
        }
    }

    rooms = {
        "NEU_R1": {
            "busy": [{"begin": 12, "end": 13}],
            "facility": "NEU",
            "capacity": 4
        },
        "NEU_R2": {
            "busy": [{"begin": 7, "end": 8}, {"begin": 16, "end": 18}],
            "facility": "NEU",
            "capacity": 6
        },
        "FTU_R1": {
            "busy": [{"begin": 11, "end": 13}],
            "facility": "FTU",
            "capacity": 3
        },
        "FTU_R2": {
            "busy": [{"begin": 17, "end": 19}],
            "facility": "FTU",
            "capacity": 8
        }
    }

    students = {
        "alice": {"missed": {"Math": 3, "English": 1}, "registered_sessions": 3},
        "bob": {"missed": {"Physics": 2}, "registered_sessions": 3},
        "charlie": {"missed": {"Math": 1, "English": 2}, "registered_sessions": 3},
        "david": {"missed": {"Chemistry": 1}, "registered_sessions": 3},
        "eva": {"missed": {}, "registered_sessions": 2},
        "frank": {"missed": {"Math": 4}, "registered_sessions": 1},
        "grace": {"missed": {"Math": 2}, "registered_sessions": 1},
        "henry": {"missed": {"English": 3, "Chemistry": 2}, "registered_sessions": 2},
        "iris": {"missed": {"English": 1, "Chemistry": 3}, "registered_sessions": 2},
        "jack": {"missed": {"English": 2}, "registered_sessions": 1},
        "kate": {"missed": {}, "registered_sessions": 1}
    }

    run_scheduler(sessions, rooms, students)


if __name__ == "__main__":
    main()

=== 🔍 Khởi tạo solver ===

=== 🏫 Tạo danh sách lớp cho mỗi session ===
Session SS1 -> Lớp: ['SS1_C0', 'SS1_C1']
Session SS2 -> Lớp: ['SS2_C0']
Session SS3 -> Lớp: ['SS3_C0', 'SS3_C1']
Session SS4 -> Lớp: ['SS4_C0']

=== 🔧 Tạo biến quyết định ===
Số biến is_class_open: 6
Số biến is_room_assigned_to_class: 12
Số biến is_student_assigned_to_class: 35

=== 📏 Thêm ràng buộc ===

=== 🎯 Xây dựng hàm mục tiêu ===

=== 🚀 Bắt đầu giải bài toán ===

✅ Lời giải tối ưu đã được tìm thấy.

              📊 BÁO CÁO PHÂN TÍCH QUYẾT ĐỊNH CHI TIẾT CHO TỪNG LỚP              

--- Lớp: SS1_C0 (Math) ---
  - Trạng thái: ❌ BỊ ĐÓNG
  - Sĩ số tiềm năng: 7 sinh viên (Yêu cầu tối thiểu: 3)
  - Phân tích kinh tế (Giả định):
    - Lợi ích tiềm năng tối đa: 827
    - Chi phí mở lớp:          -150
    - Chênh lệch tiềm năng:     677
  - Phân tích lý do:
    - Lý do chính: [LỚP THAY THẾ] Nhu cầu cho session này đã được đáp ứng bởi lớp
      'SS1_C1', do đó không cần mở thêm lớp này để tránh lãng phí.
-----------------